In [1]:
import pandas as pd
import numpy as np
import scipy.stats
import math
import warnings
warnings.filterwarnings("ignore")


In [2]:
# tmp0_gr=tmp0.groupby(['msa','statefip'],as_index=False).agg({'id':sum})
# bmk0_gr=bmk0.groupby(['msa'],as_index=False).agg({'id':sum})
# msaid_state_lookup=tmp0_gr[['msa','statefip']].merge(bmk0_gr[['msa']],on='msa').groupby(['msa','statefip'],as_index=False).sum()
# msaid_state_lookup.to_excel('/Users/hannahkamen/Downloads/msaid_state_lookup.xlsx')

#####get MSA Identifier information
# msa_id=pd.read_stata('/Users/hannahkamen/Downloads/population-migration-master/estimation/1_main_specification/acs5yr0610/dta/msa_identifier.dta')
# msa_vars=pd.read_stata('/Users/hannahkamen/Downloads/population-migration-master/estimation/1_main_specification/acs5yr0610/dta/second_stage_dataset_cl.dta')
state_split=pd.read_excel('/Users/hannahkamen/Downloads/state_pop_educ_shares.xlsx')
sl=pd.read_excel('/Users/hannahkamen/Downloads/statelookup2.xlsx')

##### (2) get coordinate data

In [3]:
###prepare dataset with centroid of every state

map_dta1=pd.read_csv('/Users/hannahkamen/Downloads/census_texas.csv')
map_dta2=pd.read_csv('/Users/hannahkamen/Downloads/census_ny.csv')
map_dta=map_dta1.append(map_dta2).reset_index()

###rename data
map_dta['centroid2']=map_dta['centroid1']+","+map_dta['centroid2']
map_dta['centroid1']=map_dta['FULL1_NAME']+","+map_dta['FULL2_NAME']
map_dta['FULL1_NAME']=map_dta['GEOID1']
map_dta['FULL2_NAME']=map_dta['GEOID2']

del map_dta['GEOID1']
del map_dta['GEOID2']


map_dta['GEOID1']=map_dta['level_1']
map_dta['GEOID2']=map_dta['Unnamed: 0']
map_dta=map_dta[['GEOID1','GEOID2','FULL1_NAME','FULL2_NAME','centroid1','centroid2']]
####get abbreviations

states=list(sl['abbrev'].unique())

state_coords=pd.DataFrame()
for s in states:
    tmp=map_dta[map_dta['FULL2_NAME'].str.contains(s)]
    tmp['state']=s
    state_coords=state_coords.append(tmp)
state_coords=state_coords.drop_duplicates(subset='state')
state_coords=state_coords[['state','FULL2_NAME','centroid2']]

In [4]:
state_coords['id']= np.arange(len(state_coords))
state_coords['name']=state_coords['state']
state_coords['lat']=state_coords['centroid2'].apply(lambda x: x.split(',')[1].replace(")",'').strip())
state_coords['lon']=state_coords['centroid2'].apply(lambda x: x.split(',')[0].replace("(",'').replace("c",'').strip())
state_coords_lm=state_coords[['id','name','lat','lon']]
state_coords_lm.to_csv('/Users/hannahkamen/Downloads/flowmap_location_lookup.csv',index=False)


##### (2) map out migration flows

In [5]:
#####import state pop shares by skilled and unskilled
state_educ=pd.read_excel('/Users/hannahkamen/Downloads/state_educ_shares.xlsx')
del state_educ['statefip']
state_educ=state_educ.merge(sl,on='state',how='inner')
state_educ=state_educ[['statefip','skl','unskl','state','abbrev','state_pop']]

In [6]:
###these are the population shares changes from the first cycle of GAMS data inputs (after initial logit shock with temp)
###the shares caluclated here represent the population change from the previous iteration 
state_split=pd.read_excel('/Users/hannahkamen/Downloads/state_pop_educ_shares.xlsx')

msaid_state_lookup=pd.read_excel('/Users/hannahkamen/Downloads/msaid_state_lookup.xlsx')
sl=pd.read_excel('/Users/hannahkamen/Downloads/statelookup2.xlsx')
tmp0=pd.read_stata('/Users/hannahkamen/Downloads/population-migration-master/estimation/1_main_specification/acs5yr0610/dta/projection_data_age2_01_wbmk_v2_1_tag.dta')
tmp0=tmp0.merge(sl,on='statefip')



In [7]:
####store in csv

master_map_unskl2=tmp0[tmp0['coll']==0]
# master_map_skl2=tmp0[tmp0['coll']==1]
del tmp0

master_map_unskl2.to_csv('/Users/hannahkamen/Downloads/map_data_unskl2_it1_tag.csv')
del master_map_unskl2
# master_map_skl2.to_csv('/Users/hannahkamen/Downloads/map_data_skl2_it1.csv')
# del master_map_skl2


In [8]:
master_map_unskl2=pd.read_csv('/Users/hannahkamen/Downloads/map_data_unskl2_it1_tag.csv')
# master_map_skl2=pd.read_csv('/Users/hannahkamen/Downloads/map_data_skl2_it1.csv')


In [9]:
master_map_unskl2=master_map_unskl2.rename(columns={'state_x':'state','abbrev_x':'abbrev'})
# master_map_skl2=master_map_skl2.rename(columns={'state_x':'state','abbrev_x':'abbrev'})

In [33]:
list(master_map_unskl2)

['Unnamed: 0',
 'id',
 'msa',
 'bpl',
 'chosen',
 'd_s',
 'd_r1',
 'd_r2',
 'inc_hat',
 'coll',
 'hot_ca',
 'cold_ca',
 'hot_w',
 'cold_w',
 'hot_mw',
 'cold_mw',
 'hot_ne',
 'cold_ne',
 'cold_coll',
 'hot_coll',
 'd_s_kid',
 'd_r1_kid',
 'd_r2_kid',
 'htem',
 'age',
 '_b_lnrho',
 '_b_fcold_ca',
 '_b_fcold_coll',
 '_b_fcold_mw',
 '_b_fcold_ne',
 '_b_fhot_ca',
 '_b_fhot_coll',
 '_b_fhot_mw',
 '_b_fhot_ne',
 '_b_fhot_w',
 '_b_d_r1',
 '_b_d_r2',
 '_b_d_r2_kid',
 '_b_d_s',
 '_b_d_s_kid',
 '_b_inc_hat',
 '_b_fcold',
 '_b_fhot',
 '_b_lnavgtemp',
 '_b_lnlake',
 '_b_lnlake2',
 '_b_lnpop',
 '_b_lnsea',
 '_b_lnsea2',
 '_b_lnslope',
 'index',
 'regionfe',
 'statefip',
 'lnrho',
 'metarea',
 'hot',
 'cold',
 'avgtemp',
 'precip',
 'dewpt',
 'relhum',
 'sun',
 'lnsea',
 'lnsea2',
 'lnlake',
 'lnlake2',
 'lnslope',
 'lnden',
 'lnhsgrad',
 'lncoll',
 'lngrad',
 'lnage',
 'lnhisp',
 'lnblack',
 'lnprec',
 'lndewpt',
 'lnrelhum',
 'lnsun',
 'lnavgtemp',
 'lnpop',
 'lnmanu',
 'lngvt',
 'lnpropt',
 'lnin

#### Unskilled Map

In [35]:
###### prep data for map

#get sum of shares in all other states for people who chose states 1-44
#get ids that live in state s currentlly
unskl=pd.DataFrame()
for s in master_map_unskl2['state'].unique():
    living_now=[]
    tmp1=master_map_unskl2[(master_map_unskl2['state']==s) & (master_map_unskl2['chosen']==1)]
    ####list of ids currently living in state s
    living_now.append(list(tmp1['id'].unique())[0])
    ###limit dataframe to the ids of people living in state s
    tmp2=master_map_unskl2[master_map_unskl2['id'].isin(living_now)]
    ####now groupby shares across all states
    tmp3=tmp2.groupby(['state'],as_index=False).agg({'share1':sum,'share0':sum})
    ###tag origin state
    tmp3['living_flag']=s
    unskl=unskl.append(tmp3)
    
######merge with state populaiton
unskl=unskl.merge(state_split,left_on='living_flag',right_on='state',how='inner')
unskl=unskl.rename(columns={'state_x':'moving_to'})

######get share difference between current  iteration and benchmark
unskl['share_diff']=unskl['share1']-unskl['share0']

In [40]:
####read in pct changes by state and education
le0=pd.read_csv('/Users/hannahkamen/Downloads/le0_shock1_v2_test2_adj_d2_tag.csv')

le0=le0[['r','sk','skill_shr']]
le0=le0.drop_duplicates(subset=['r','sk'])
le0=le0.rename(columns={'sk':'educ_id','skill_shr':'final_change','r':'abbrev'})
le0=le0.merge(sl,on='abbrev')
le0_skl=le0[le0['educ_id']=='skl']
le0_unskl=le0[le0['educ_id']=='unskl']

In [43]:
#####prepare pop map data for skilled entrance
all_unskl_entrance=pd.DataFrame()

for s in le0_unskl[le0_unskl['final_change']>0]['state'].unique():
    ###for each entrance state, calculate shares by origin state that make up total change in the positive share diff
    master_map_unskl_lm=unskl[((unskl['moving_to']==s) & (unskl['share_diff']>0))]
    #####get totals 
    master_map_unskl_lm_totalinflows=master_map_unskl_lm.groupby('moving_to').agg({'share_diff':sum})
    ###rename and merge
    master_map_unskl_lm_totalinflows=master_map_unskl_lm_totalinflows.rename(columns={'share_diff':'share_diff_tot'})
    master_map_unskl_lm=master_map_unskl_lm.merge(master_map_unskl_lm_totalinflows,on='moving_to')
    
    master_map_unskl_lm['inflow_share']= master_map_unskl_lm['share_diff']/master_map_unskl_lm['share_diff_tot']
    
    master_map_unskl_lm=master_map_unskl_lm[['moving_to','living_flag','inflow_share','share_diff_tot']]
    all_unskl_entrance=all_unskl_entrance.append(master_map_unskl_lm)
    
all_unskl_entrance=all_unskl_entrance.merge(state_educ,left_on='moving_to',right_on='state').merge(le0_unskl,on='state')
all_unskl_entrance['raw_pop_change_moving_to']=all_unskl_entrance['final_change']*all_unskl_entrance['state_pop']*all_unskl_entrance['unskl']*all_unskl_entrance['inflow_share']

##merge in pop info

all_unskl_entrance=all_unskl_entrance.rename(columns={'abbrev_x':'dest'})
all_unskl_entrance=all_unskl_entrance.merge(sl[['state','abbrev']],left_on='living_flag',right_on='state')
all_unskl_entrance=all_unskl_entrance.rename(columns={'abbrev':'origin'})

all_unskl_entrance=all_unskl_entrance.merge(state_coords_lm,left_on='dest',right_on='name')
del all_unskl_entrance['dest']
all_unskl_entrance['dest']=all_unskl_entrance['id']
del all_unskl_entrance['id']
all_unskl_entrance=all_unskl_entrance.merge(state_coords_lm,left_on='origin',right_on='name')
del all_unskl_entrance['origin']


# all_unskl_entrance=all_unskl_entrance.merge(state_educ,left_on='moving_to',right_on='state').merge(total_skilled_pop_changes,left_on='moving_to',right_on='statefip')
all_unskl_entrance['origin']=all_unskl_entrance['id']
all_unskl_entrance['count']=all_unskl_entrance['raw_pop_change_moving_to']

all_unskl_entrance_lm=all_unskl_entrance[['origin','dest','count']]

all_unskl_entrance_lm['count']=all_unskl_entrance_lm['count']/14000000

all_unskl_entrance_lm.to_csv('/Users/hannahkamen/Downloads/unskilled_entrance2_it1_tag4.csv',index=False)



In [42]:
all_unskl_entrance.groupby('moving_to',as_index=False).agg({'count':sum})

,moving_to,count
0,arkansas,159.385578
1,colorado,3844.006451
2,connecticut,4627.313507
3,delaware,1064.624842
4,idaho,7608.633079
5,illinois,6024.137731
6,indiana,4269.719654
7,iowa,3680.247028
8,kansas,741.822506
9,kentucky,3686.018044
